In [124]:
# %load dataset.py
import re
import pickle
import numpy as np
from collections import defaultdict

from nltk.tokenize import word_tokenize

import torch
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence

UNK = '<unk>'
PAD = '<pad>'
SOS = '<s>' # start of sentence
EOS = '</s>'

BATCH_SIZE = 1000

class GloveVocabulary:
    def __init__(self, glove_vocab_path, glove_emb_path):
        self.idx_to_str = [PAD, SOS, EOS] # <unk> is in GloVe
        self.start_idx = len(self.idx_to_str) # length at which real tokens starts
        # load glove into self.idx_to_str and self.str_to_idx
        with open(glove_vocab_path, 'rb') as f:
            glove_vocab = pickle.load(f)
        with open(glove_emb_path, 'rb') as f:
            glove_emb = pickle.load(f)
        self.idx_to_str += glove_vocab
        self.str_to_idx = {s: idx for idx, s in enumerate(self.idx_to_str)}

        # TODO: initialize emb for special tokens
        # instead of random vector, use the mean of all glove vectors for special tokens
        glove_emb = torch.tensor(glove_emb)
        mean_vec = glove_emb.mean(dim=0, keepdim=True)
        self.embedding = torch.cat(
            [mean_vec, mean_vec, mean_vec, glove_emb], dim=0
        )

    def __len__(self):
        return len(self.idx_to_str)

    @staticmethod
    def tokenize(line):
        # TODO: try different tokenizers
        return word_tokenize(line.lower())

    def numericalize(self, line):
        """
        Call this only after the vocab has been built
        """
        tokens = self.tokenize(line)
        ret = [self.str_to_idx[SOS]]
        for token in tokens:
            if token in self.str_to_idx:
                ret.append(self.str_to_idx[token])
            else:
                ret.append(self.str_to_idx[UNK])
        ret.append(self.str_to_idx[EOS])
        return torch.LongTensor(ret)

    def denumericalize(self, token_indices):
        """
        Invert numericalize, returns a string
        """
        # remove start and end token
        ret = []
        for idx in token_indices[1 : -1]:
            token = self.idx_to_str[idx]
            # break early when hitting <pad> token
            if token == PAD:
                break
            else:
                ret.append(token)
        return ' '.join(ret)

class TrainDataset(Dataset):
    def __init__(self, train_path, glove_vocab_path, glove_emb_path, num_transforms=3):
        """
        num_transforms: number of transforms to apply to the line to generate a negative sample
        """
        self.num_transforms = num_transforms
        self.first_column_lines = [] # lines in the first column
        self.second_column_lines = []
        with open(train_path, 'rt') as f:
            for line in f:
                # do minimal amount of preprocessing here, lowercasing is done in vocab
                first, second = line.split('\t')
                self.first_column_lines.append(first)
                self.second_column_lines.append(second)

        self.vocab = GloveVocabulary(glove_vocab_path, glove_emb_path)

    def __len__(self):
        return len(self.first_column_lines)

    def generate_negative_example(self, numericalized_line):
        """
        numericalized_line: torch.LongTensor
        """
        # randomly substitute in words after vocab.start_idx
        # TODO: insertion, deletion, permutation
        ret = numericalized_line.detach().clone()
        # position in line to perturb
        token_indices = np.random.choice(range(len(numericalized_line)),
        self.num_transforms, replace=False)
        # the last token is <unk>
        vocab_indices = np.random.choice(range(self.vocab.start_idx, len(self.vocab) - 1),
        self.num_transforms)
        for tok_idx, vocab_idx in zip(token_indices, vocab_indices):
            ret[tok_idx] = vocab_idx
        return ret

    def __getitem__(self, index):
        """
        Return a triplet of numericalized lines
        original, positive, and negative example
        (line, paraphrased line, non-paraphrasal line)
        """
        line = self.first_column_lines[index]
        positive_line = self.second_column_lines[index]
        # convert tokens to indices
        numericalized_line = self.vocab.numericalize(line)
        numericalized_positive = self.vocab.numericalize(positive_line)
        # generate a negative numericalized example
        numericalized_negattive = self.generate_negative_example(numericalized_line)

        return numericalized_line, numericalized_positive, numericalized_negattive

class PadCollate:
    """
    Pad lines in the same batch to the same length
    """
    def __init__(self, pad_idx):
        """
        pad_idx
        """
        self.pad_idx = pad_idx

    def __call__(self, batch):
        # use long tensor for embedding
        lines = [item[0] for item in batch]
        positives = [item[1] for item in batch]
        negatives = [item[2] for item in batch]
        lines = pad_sequence(lines, batch_first=False, padding_value=self.pad_idx)
        positives = pad_sequence(positives, batch_first=False, padding_value=self.pad_idx)
        negatives = pad_sequence(negatives, batch_first=False, padding_value=self.pad_idx)
        return lines, positives, negatives

def get_train_loader(train_path, glove_vocab_path, glove_emb_path):
    dataset = TrainDataset(train_path, glove_vocab_path, glove_emb_path)
    pad_idx = dataset.vocab.str_to_idx[PAD]
    loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True, collate_fn=PadCollate(pad_idx=pad_idx))
    return loader

def load_dev_devtest(vocab, path):
    """
    calls vocab.numericalize
    returns first_lines, second_lines, labels
    """
    first_lines = []
    second_lines = []
    labels = []
    with open(path, 'rt') as f:
        for line in f:
            chunks = line.strip().split('\t')
            first = vocab.numericalize(chunks[0])
            second = vocab.numericalize(chunks[1])
            label = int(chunks[2])
            first_lines.append(first)
            second_lines.append(second)
            labels.append(label)
    return first_lines, second_lines, labels

def load_test(vocab, path):
    """
    skip id column, calls vocab.numericalize
    """
    first_lines, second_lines = [], []
    with open(path, 'rt') as f:
        for line in f:
            chunks = line.strip().split('\t')
            first = vocab.numericalize(chunks[1])
            second = vocab.numericalize(chunks[2])
            first_lines.append(first)
            second_lines.append(second)
    return first_lines, second_lines


In [148]:
loader = get_train_loader('data/train.tsv', 'data/glove_vocab.pkl', 
                    'data/glove_emb.pkl')

In [126]:
loader.dataset.vocab.str_to_idx['hello']

13078

In [127]:
# %load model.py
import torch
import torch.nn as nn

EMB_DIM = 50

class RNNClassifier(nn.Module):
    def __init__(self, pretrained_emb=None, freeze_emb=False, vocab_size=None, emb_dim=EMB_DIM):
        """
        vocab_size must be not None if no pretrained_emb is given
        """
        super(RNNClassifier, self).__init__()
        if pretrained_emb is None:
            self.emb = nn.Embedding(vocab_size, emb_dim)
            torch.nn.init.uniform_(self.emb.weight, -0.01, 0.01)
        else:
            self.emb = nn.Embedding.from_pretrained(pretrained_emb, freeze=freeze_emb)

        rnn_input_dim = self.emb.weight.shape[1] # EMB_DIM
        rnn_output_dim = 128
        # TODO: bidirectional?
        self.rnn = nn.GRU(rnn_input_dim, rnn_output_dim, batch_first=False)

        # pass the concatenation of two RNN outputs to fully connected layers
        fc_input_dim = rnn_output_dim * 2
        fc_hidden_dim = 128
        self.fc1 = nn.Linear(fc_input_dim, fc_hidden_dim)
        self.fc2 = nn.Linear(fc_hidden_dim, 1) # output a scalar for class probability

    def forward(self, x1, x2):
        """
        x1: first sentence, x2: second setences
        (seq_len, batch_size)
        """
        x1 = self.emb(x1)
        x2 = self.emb(x2)
        hidden = None
        for token in x1:
            out1, hidden = self.rnn(token.unsqueeze(0), hidden)
        # TODO: is it better to pass hidden=hidden, hidden=output, or hidden=None
        # can do truncate or pad
        hidden = None
        for token in x2:
            out2, hidden = self.rnn(token.unsqueeze(0), hidden)
        fc_input = torch.cat([out1, out2], dim=-1).squeeze()
        out = torch.tanh(self.fc1(fc_input))
        # use sigmoid with BCELoss
        out = torch.sigmoid(self.fc2(out))
        return out


In [128]:
model = RNNClassifier(pretrained_emb=loader.dataset.vocab.embedding)

In [129]:
np.set_printoptions(precision=4)
np.random.seed(0)
torch.manual_seed(0)

In [130]:
import torch.optim as optim
from sklearn.metrics import confusion_matrix, accuracy_score

In [131]:
from tqdm.notebook import tqdm

In [149]:
BATCH_SIZE = 100

In [150]:
loss_func = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [151]:
# positive and negative labels
ones = torch.ones((BATCH_SIZE, 1)).to(device)
zeros = torch.zeros((BATCH_SIZE, 1)).to(device)

In [152]:
flat_true_labels = np.concatenate(
    [np.ones(BATCH_SIZE, dtype=int), 
     np.zeros(BATCH_SIZE, dtype=int)])

In [153]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [154]:
model = model.to(device)

In [155]:
for line, positive, negative in tqdm(loader):
    line = line.to(device)
    positive = positive.to(device)
    negative = negative.to(device)
    out_positive = model(line, positive)
    out_negative = model(line, negative)
    optimizer.zero_grad()
    loss = loss_func(out_positive, ones)
    loss += loss_func(out_negative, zeros)
    loss.backward()
    optimizer.step()
    
    preds = torch.cat([out_positive, out_negative]).cpu().flatten()
    preds = np.where(preds > 0.5, 1, 0).astype(int)
    
    print(accuracy_score(flat_true_labels, preds))

/scratch/ruolinzheng/conda/envs/py38/lib/python3.8/site-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([100])) that is different to the input size (torch.Size([100, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


0.51
0.505
0.5
0.495
0.5
0.505
0.495
0.5
0.5
0.5
0.5
0.5
0.5
0.485
0.5
0.51
0.5
0.495
0.505
0.505
0.505
0.495
0.5
0.5
0.5
0.5
0.49
0.495
0.485
0.505
0.485
0.49
0.495
0.5
0.5
0.5
0.5
0.5
0.5
0.495
0.515
0.515
0.505
0.5
0.555
0.505
0.53
0.535
0.62
0.57
0.89
0.91
0.9
0.915
0.9
0.86
0.915
0.88
0.91
0.95
0.975
0.965
0.97
0.965
0.96
0.975
0.965
0.96
0.955
0.99
0.975
0.99
0.985
0.98
0.995
0.995
0.985
0.985
0.985
0.995
1.0
0.99
1.0
0.995
0.99
1.0
1.0
1.0
0.995
0.99
1.0
0.995
1.0
0.985
0.985
0.985
0.98
0.99
1.0
0.995
0.995
1.0
1.0
0.995
0.985
0.995
1.0
0.985
1.0
0.99
1.0
1.0
0.995
0.995
0.995
0.99
0.995
0.99
0.99
0.975
0.98
0.96
0.995
0.975
1.0
0.995
1.0
0.995
0.995
0.995
0.99
0.99
0.995
0.995
0.99
0.99
0.995
0.995
1.0
0.995
0.995
0.99
0.99
0.99
0.995
0.995
1.0
0.995
1.0
0.995
1.0
0.995
0.98
1.0
0.99
0.995
0.995
0.99
0.99
0.995
0.995
0.985
0.985
0.99
1.0
0.995
1.0
0.99
0.995
0.99
0.985
1.0
0.985
0.99
0.99
0.995
0.99
1.0
0.99
0.99
1.0
0.985
0.99
1.0
0.995
0.995
0.995
1.0
1.0
0.995
0.99
1.0
0.995

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [96]:
vocab = loader.dataset.vocab

In [97]:
dev = load_dev_devtest(vocab, 'data/dev+devtest/dev.tsv')
devtest = load_dev_devtest(vocab, 'data/dev+devtest/devtest.tsv')

In [156]:
print('hello')

hello


In [100]:
dev[0][0].shape

torch.Size([12])

In [162]:
dev[2][0]

1

In [165]:
model(dev[0][0].unsqueeze(1).cuda(), 
      dev[1][0].unsqueeze(1).cuda()).cpu().numpy()

RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.

In [170]:
def eval(model, data):
    model.eval()
    first_lines, second_lines, labels = data
    preds = np.empty(len(first_lines))
    with torch.no_grad():
        for idx, tup in enumerate(zip(first_lines, second_lines)):
            first, second = tup
            first = first.to(device)
            second = second.to(device)
            # unsqueeze in batch_size
            pred = model(first.unsqueeze(1), 
                         second.unsqueeze(1)).cpu().numpy()
            preds[idx] = pred
    preds = np.where(preds > 0.5, 1, 0)
    return preds

In [171]:
preds = eval(model, dev)

In [172]:
dev[2][:10]

[1, 1, 1, 1, 1, 1, 0, 0, 0, 0]

In [173]:
accuracy_score(preds, dev[2])

0.628361858190709

In [174]:
confusion_matrix(dev[2], preds)

array([[  1, 300],
       [  4, 513]])